<a href="https://colab.research.google.com/github/delemarchand2020/TestsAzureAI/blob/main/PoC_0_shot_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classification avec AzureVision OCR + Embedding Azure AI (OpenAI) + Cos-similarité
https://medium.com/@jh.baek.sd/concised-vector-embeddings-for-beginners-guide-3907793e7b10

Autre solution mais impliquant des LLM et voici pourquoi:
* https://levelup.gitconnected.com/advanced-document-classification-with-llms-8801eaee3c58

##Installation des librairies

In [ ]:
!pip install -q azure-ai-vision-imageanalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 585.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s e

In [ ]:
!pip install -q pdf2image

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 1s (259 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install -q docx2pdf

In [ ]:
!pip install -q openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Chargement des clés Azure et OpenAI

In [ ]:
VISION_ENDPOINT = "https://demo-desj-azure-ai.cognitiveservices.azure.com/"

In [ ]:
# Chemin du fichier après le montage de Google Drive
file_path = "/content/gdrive/MyDrive/key.openai.txt"

# Lire le contenu du fichier
with open(file_path, 'r') as file:
    content = file.read()

# Configuration initiale de l'API OpenAI
openai_api_key = content

In [ ]:
# Chemin du fichier après le montage de Google Drive
file_path = "/content/gdrive/MyDrive/key.azure.txt"

# Lire le contenu du fichier
with open(file_path, 'r') as file:
    content = file.read()

# Configuration initiale de l'API Azure
VISION_KEY = content

##Chargement d'images de test

In [ ]:
! ls -l "/content/gdrive/MyDrive/recu_masso_photo.jpg"

-rw------- 1 root root 676809 Apr 21 21:12 /content/gdrive/MyDrive/recu_masso_photo.jpg


In [ ]:
!cp "/content/gdrive/MyDrive/recu_masso_photo.jpg" .
!cp "/content/gdrive/MyDrive/McGill_Enrolment_Letter_2023-2024.pdf" .
!cp "/content/gdrive/MyDrive/releve_notes.pdf" .
!cp "/content/gdrive/MyDrive/billet_medecin.jpg" .
!cp "/content/gdrive/MyDrive/fac_ortho.jpg" .

##Fonctions utiles

###Conversion PDF ou DOCX en image

In [ ]:
from pdf2image import convert_from_path
import os

def convert_pdf_to_jpg(file_path):
    # Vérifier si le fichier existe
    if not os.path.isfile(file_path):
        raise FileNotFoundError("Le fichier spécifié n'existe pas.")

    # Vérifier si le fichier est un fichier PDF
    if not file_path.lower().endswith('.pdf'):
        return file_path

    # Chemin du fichier JPEG de sortie
    jpg_path = os.path.splitext(file_path)[0] + ".jpg"

    # Convertir le PDF en une liste d'images PIL
    images = convert_from_path(file_path)

    # Sauvegarder la première image en tant que fichier JPEG
    images[0].save(jpg_path, "JPEG")

    # Retourner le chemin du fichier JPEG
    return jpg_path

In [ ]:
from docx2pdf import convert

def convert_docx_to_pdf(file_path):
    # Vérifier si le fichier existe
    if not os.path.isfile(file_path):
        raise FileNotFoundError("Le fichier spécifié n'existe pas.")

    # Vérifier si le fichier est un fichier DOCX
    if not file_path.lower().endswith('.docx'):
        return file_path

    # Chemin du fichier PDF de sortie
    pdf_path = os.path.splitext(file_path)[0] + ".pdf"

    # Convertir le fichier DOCX en PDF
    convert(file_path, pdf_path)

    # Retourner le chemin du fichier PDF
    return convert_pdf_to_jpg(pdf_path)


In [ ]:
from PIL import Image
import io

def get_PIL_image(img_path):
  with open(img_path, 'rb') as image_file:
      content = image_file.read()
  return Image.open(io.BytesIO(content))

In [ ]:
import numpy as np

def convert_cos_sim_to_angle(cos_sim):
    # Calcul de l'angle en radians
    angle_rad = np.arccos(cos_sim)

    # Conversion de radians en degrés
    angle_deg = np.degrees(angle_rad)

    return round(angle_deg,1)

###Pre-traitement textes

In [ ]:
import re

# Définir les expressions régulières pour les dates, les prix, les mots français (y compris les accents) et les nombres
date_regex = re.compile(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b|\b\d{1,2}-\d{1,2}-\d{2,4}\b|\b\d{4}-\d{2}-\d{2}\b|\b\d{2,4}/\d{2}/\d{2,4}\b')
price_regex = re.compile(r'\b\d+[\.,]?\d*\s?(€|EUR|euros?|dollars?|USD|CHF)\b')
french_word_regex = re.compile(r'\b[a-zA-ZàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ]+\b')
number_regex = re.compile(r'\b\d+\b')

# Définir une expression régulière pour les éléments indésirables (ponctuation et autres caractères non désirés)
unwanted_regex = re.compile(r'[^\w\s€|EUR|euros|dollars|USD|CHFàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ\'/-]')

# Fonction pour filtrer le texte
def filter_text(text):
    filtered_elements = []
    words = text.split()

    for word in words:
        # Supprimer les signes de ponctuation et autres caractères non désirés
        clean_word = unwanted_regex.sub('', word)

        if date_regex.match(clean_word) or price_regex.match(clean_word) or french_word_regex.match(clean_word) or number_regex.match(clean_word):
            filtered_elements.append(clean_word)

    return ' '.join(filtered_elements)

In [ ]:
# @title Test pré-traitement
# Exemple d'utilisation
text = "Bonjour, je veux acheter une voiture le 12/06/2024 pour 1500 EUR. Mon chat aime bien jouer avec la souris. La date d'aujourd'hui est 2024-06-10. J'ai 2 chats et 1 chien."
filtered_text = filter_text(text)
print(filtered_text)

Bonjour je veux acheter une voiture le 12/06/2024 pour 1500 EUR Mon chat aime bien jouer avec la souris La date d'aujourd'hui est 2024-06-10 J'ai 2 chats et 1 chien


###Appels Azure Vision
https://learn.microsoft.com/fr-ca/python/api/overview/azure/ai-vision-imageanalysis-readme?view=azure-python-preview

In [ ]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

# Set the values of your computer vision endpoint and computer vision key
# as environment variables:
try:
    endpoint = VISION_ENDPOINT
    key = VISION_KEY
except KeyError:
    print("Missing environment variable 'VISION_ENDPOINT' or 'VISION_KEY'")
    print("Set them before running this sample.")
    exit()

# Create an Image Analysis client for synchronous operations
client = ImageAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

In [ ]:
# @title Test AzureVision
test=True

if test:
    # Load image to analyze into a 'bytes' object
    with open("billet_medecin.jpg", "rb") as f:
        image_data = f.read()

    # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
    result = client.analyze(
        image_data=image_data,
        visual_features=[VisualFeatures.READ, VisualFeatures.OBJECTS, VisualFeatures.TAGS]
    )

    # Print text (OCR) analysis results to the console
    #print("Image analysis results:")
    #print(" Read:")
    #if result.read is not None:
    #  if result.read.blocks != []:
    #    for line in result.read.blocks[0].lines:
    #        print(f"   Line: '{line.text}', Bounding box {line.bounding_polygon}")
    #        for word in line.words:
    #            print(f"     Word: '{word.text}', Bounding polygon {word.bounding_polygon}, Confidence {word.confidence:.4f}")
    if result.tags is not None:
      print(" Tags:")
      for o in result.tags.list:
          print(f"   tag: '{o}'")

 Tags:
   tag: '{'name': 'text', 'confidence': 0.9999885559082031}'
   tag: '{'name': 'handwriting', 'confidence': 0.9781054854393005}'
   tag: '{'name': 'ink', 'confidence': 0.9036844968795776}'
   tag: '{'name': 'paper', 'confidence': 0.9036161303520203}'
   tag: '{'name': 'letter', 'confidence': 0.8745497465133667}'
   tag: '{'name': 'paper product', 'confidence': 0.862065315246582}'
   tag: '{'name': 'calligraphy', 'confidence': 0.8448853492736816}'
   tag: '{'name': 'document', 'confidence': 0.5845134258270264}'


In [ ]:
# Get text (OCR) analysis results
def get_all_words(result):
    all_words = []
    if result.read.blocks != []:
      for line in result.read.blocks[0].lines:
          for word in line.words:
              all_words.append(word.text)
    return all_words

def get_all_tags(result):
    all_tags = []
    if result.tags is not None:
      all_tags = result.tags.list
    return all_tags

if test:
  print(get_all_words(result))
  print(get_all_tags(result))

['HYGEA', 'clinique', 'médicale', '5100', 'boul.', 'de', 'Maisonneuve', 'O.', 'Bureau', '710', 'Tél.', ':', '514-938-0995', 'Montréal', '(Qc)', 'H4A', '3T2', 'Fax', ':', '514-938-4476', 'Date:', 'Sul', '31123', 'Marina', 'Grandin', 'Nom', ':', 'Addresse', ':', 'R', 'Custom', 'orthotics', 'Dx', 'Hallux', 'Valgus', 'Geld', 'sman', '87582']
[{'name': 'text', 'confidence': 0.9999885559082031}, {'name': 'handwriting', 'confidence': 0.9781054854393005}, {'name': 'ink', 'confidence': 0.9036844968795776}, {'name': 'paper', 'confidence': 0.9036161303520203}, {'name': 'letter', 'confidence': 0.8745497465133667}, {'name': 'paper product', 'confidence': 0.862065315246582}, {'name': 'calligraphy', 'confidence': 0.8448853492736816}, {'name': 'document', 'confidence': 0.5845134258270264}]


In [ ]:
# Initialize an empty dictionary to store the extracted text
text_dict = {}
tags_dict = {}

def document_texts_detection(img_path, debug=False):
    # Check if the image has already been seen
    if img_path not in text_dict:
        # Load image to analyze into a 'bytes' object
        with open(img_path, "rb") as f:
            image_data = f.read()

        # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
        result = client.analyze(
            image_data=image_data,
            visual_features=[VisualFeatures.READ, VisualFeatures.TAGS]
        )
        # Store the extracted text in the dictionary
        text_dict[img_path] = get_all_words(result)
        tags_dict[img_path] = get_all_tags(result)

    # Return the extracted text from the dictionary
    return text_dict[img_path], tags_dict[img_path]


###Appels OpenAI pour embeddings
https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python

In [ ]:
from openai import OpenAI
import openai

openai.api_key = openai_api_key

client_openai = OpenAI(api_key=openai.api_key)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize an empty dictionary to store the embeddings
embeddings_dict = {}

def get_text_embedding(input, model="text-embedding-3-small"):

    if input != None and input != "":
        # Check if the input has already been seen
        if input not in embeddings_dict:
            embeddings_batch_response_openAI = client_openai.embeddings.create(input = [input], model=model)
            # Store the embedding in the dictionary
            embeddings_dict[input] = np.array([embeddings_batch_response_openAI.data[0].embedding])
        # Return the embedding from the dictionary
        return embeddings_dict[input]
    else:
        return np.zeros((1, 1536))

###Tests des appels

In [ ]:
texts = document_texts_detection(img_path="recu_masso_photo.jpg", debug=False)

In [ ]:
print(f"Nombre de mots/token détectés: {len(texts)}")

Nombre de mots/token détectés: 2


In [ ]:
# @title Texte pour test
texte = """
Université
de Montréal
Denis Lemarchand
4958 avenue grosvenor
montreal QC H3W 2M1
Canada
Programme d'études:
Spécialisation:
Relevé de notes non officiel
Relevé des études de cycles supérieurs
Automne 2022
217510 Informatique (Maîtrise)
Apprentissage automatique (Spécialisation)
Date d'émission:
Matricule:
CPER QC:
2024-01-25
20241211
LEMD02036912
Cours
IFT 6390
Description
Crédits:
Apprent. machine: fondements
Suivis
4.000
Obtenus
4.000
Note Points
A
16.000
Moy.gr.
2-175-1-0 72 Cours à option
Crédits:
Suivis
Moyenne générale trimestrielle
4.000 Total du trimestre
4.000
Obtenus
4.000
Moy.
4.000
Points
16.000
Hiver 2023
Programme d'études:
Spécialisation:
217510 Informatique (Maîtrise)
Apprentissage automatique (Spécialisation)
Cours
Description
Crédits:
IFT 6135
Aprentissage de représent.
Suivis
4.000
Obtenus
4.000
Note Points
A-
14.800
Moy.gr.
2-175-1-0 72 Cours à option
Crédits:
Moyenne générale trimestrielle
3.700 Total du trimestre
Suivis
4.000
Obtenus
4.000
Moy.
4.000
Points
14.800
Automne 2023
Programme d'études:
Spécialisation:
217510 Informatique (Maîtrise)
Apprentissage automatique (Spécialisation)
Cours
Description
Crédits:
IFT 6758
Science des données
Suivis
4.000
Obtenus
4.000
Note Points
A
16.000
Moy.gr.
2-175-1-0 72 Cours à option
Crédits:
Suivis
Moyenne générale trimestrielle
4.000 Total du trimestre
4.000
Obtenus
4.000
Moy.
4.000
Points
16.000
Hiver 2024
Programme d'études:
Spécialisation:
217510 Informatique (Maîtrise)
Intelligence artificielle (Spécialisation)
Cours
Description
Crédits:
IFT 6261
Traitement des connaissances
2-175-1-0 72 Cours à option
Suivis
Obtenus Note
Points
Moy.gr.
Pour être officiel, le relevé de notes doit porter le sceau de l'Université et la signature du Registraire.
Page 1 de 2
"""

###Tests similarité

In [ ]:
texte1 = "recevoir une paye pour son métier"
texte2 = "une bulletin de salaire emploi ou travail ou profession ou tâche"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.51349919]])

In [ ]:
texte1 = "recevoir une peine pour son crime"
texte2 = "une bulletin de salaire emploi ou travail ou profession ou tâche"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.21518634]])

In [ ]:
texte1 = "recevoir une peine pour un crime"
texte2 = "application d'une décision de jugement pénal"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.5661862]])

In [ ]:
texte1 = "recevoir une paye pour son métier"
texte2 = "application d'une décision de jugement pénal"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.29382311]])

In [ ]:
def my_cosine_similarity(vector1, vector2):
    #Produit scalaire des vecteurs
    scalar_product = np.dot(vector1, vector2.T)
    #Norme euclidienne des vecteurs
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    #Expression analytique du cosinus dans un espace euclidien
    cosine = scalar_product / (norm_vector1 * norm_vector2)
    return cosine

my_cosine_similarity(vector1, vector2)

array([[0.29382311]])

In [ ]:
#text-embedding-ada-002, text-embedding-3-small, text-embedding-3-large
model="text-embedding-3-large"
cosine_similarity(get_text_embedding(texte1, model=model), get_text_embedding(texte2, model=model))

array([[0.29382311]])

##Classification

In [ ]:
# Define the mapping of sub-types to types
type_mapping = {
    "Relevé de notes officiel universitaire": "Preuve de scolarité",
    "Relevé de notes non officiel universitaire": "Preuve de scolarité",
    "Facture session universitaire": "Preuve de scolarité",
    "Horaire de cours": "Preuve de scolarité",
    "Bulletin cumulatif de notes": "Preuve de scolarité",

    "Certificat médical": "Billet du médecin",
    "Formulaire d'arrêt de travail": "Billet du médecin",
    "Billet du medecin": "Billet du médecin",
    "Prescription médicale": "Billet du médecin",
    "Ordonnance médicale clinique": "Billet du médecin",

    "Spécimen de chèque": "Spécimen de chèque",
    "Payer à l'ordre de": "Spécimen de chèque",
    "100 dollars": "Spécimen de chèque",
    "Banque": "Spécimen de chèque",
    "MICR code": "Spécimen de chèque",

    "Facture soins dentaires": "Facture",
    "Facture soin thérapeutique": "Facture",
    "Facture ou reçu soin massage thérapeutique": "Facture",
    "Facture et services": "Facture"
}

# Derive types_doc from type_mapping
sous_types_doc = list(type_mapping.keys())
sous_types_doc_original = sous_types_doc.copy()
type_mapping_original = type_mapping.copy()

In [ ]:
import json

def classify_text(texte, sous_types_doc, seuil=0.4):
    classification = {}
    insert_autre = False
    total_score = 0

    # Calcul de la similarité pour chaque type de document
    for sous_type in sous_types_doc:
        similarity_score = cosine_similarity(get_text_embedding(texte), get_text_embedding(sous_type))[0][0]
        if similarity_score >= seuil:
            doc_type = type_mapping[sous_type]
            if doc_type in classification:
                classification[doc_type]["cumul des scores de similarité"] += similarity_score
            else:
                classification[doc_type] = {"cumul des scores de similarité": similarity_score}
            total_score += similarity_score
        elif not insert_autre:
            if "Autre" in classification:
                classification["Autre"]["cumul des scores de similarité"] += seuil
            else:
                classification["Autre"] = {"cumul des scores de similarité": seuil}
            total_score += seuil
            insert_autre = True

    # Calcul du pourcentage de chaque score de similarité
    for doc_type, item in classification.items():
        item["part dans la classification (%)"] = round((item["cumul des scores de similarité"] / total_score) * 100, 2)

    # Tri des résultats par score de similarité en ordre décroissant
    sorted_classification = sorted(classification.items(), key=lambda x: x[1]["cumul des scores de similarité"], reverse=True)

    # Conversion de la liste triée en format JSON
    return json.dumps(sorted_classification, indent=4)

In [ ]:
# @title Test classification
classification_sorted = classify_text(filter_text(texte), sous_types_doc, seuil=0.45)
print(classification_sorted)

[
    [
        "Preuve de scolarit\u00e9",
        {
            "cumul des scores de similarit\u00e9": 1.6311968816563556,
            "part dans la classification (%)": 78.38
        }
    ],
    [
        "Autre",
        {
            "cumul des scores de similarit\u00e9": 0.45,
            "part dans la classification (%)": 21.62
        }
    ]
]


##Interface UI pour la démo

In [ ]:
def process_document(img_path, new_type_doc, categorie, parametres, seuil):
    global sous_types_doc_original, type_mapping_original, sous_types_doc, type_mapping

    img_path = convert_pdf_to_jpg(img_path)
    img_path = convert_docx_to_pdf(img_path)
    texts, tags = document_texts_detection(img_path=img_path)
    texts = " ".join(texts)
    image = get_PIL_image(img_path=img_path)

    if "Réinitialiser la liste des sous-types/types" in parametres:
        types_doc_etendu = sous_types_doc_original.copy()
        sous_types_doc = sous_types_doc_original.copy()
        type_mapping = type_mapping_original.copy()
    else:
        types_doc_etendu = sous_types_doc.copy()
        if new_type_doc != "":
            if new_type_doc not in sous_types_doc:
                types_doc_etendu.append(new_type_doc)
                # Add the new type to the type_mapping dictionary
                type_mapping[new_type_doc] = categorie

    if "Nettoyage Texte" in parametres:
        texts = filter_text(texts)

    analyse = classify_text(texts, types_doc_etendu, seuil=seuil)

    return image, analyse, texts, tags, type_mapping

In [ ]:
import gradio as gr

# URL de l'image du diagramme
#diagram_url = "https://diagrams.helpful.dev/d/d:P8TaRIVr-png-base-64-for-mobile"
diagram_url = "https://mermaid.ink/img/pako:eNp9U0FOwzAQ_MrKF0CiPCCHSqj0gIRUiSJOuSz2ElZK7GBvKgriQX1HP8Y6TUtpgFxie2cmuzPxh7HBkSlMoteOvKUbxipiU3rQp8UobLlFL3ATbNeQLjDB7AVjRf3OdeCGyphz22BFPSH4FcXEwQN54Hw8Ri9m9xk7f5OIVjJWxYXehOD8-r2LBI-cJS7G1HnzRM6xr3qB1KIlWJGVKzhftOSvb78Rv7BnIbGnJTdcY2RZ7zpuFIKctA1dQNpXt5u_vXlYt5Qy-46Tdu3OthsfvN1uEjgC0Wp-7_1KY6F7Sl0tRxISebuhcQc76v67k-m0t7oYkolQ00ksfV1xanIxxCEcT9LQokIOVhW7LFid38VAAqXJvobIiWJpfrbRj_9ToDSPR3A49iNlQ0IdKqbBiQNPNU4zKYZEdLZshg1pMorklDNRncHRApahH7hGqP911lyahmKD7PRWfGTh0siL_uylyeM4ekYVzKN_KhQ7Ccu1t6aQ2NGl6VqHsr9EpnjGOh1O547VieHw8wur_0EX?type=png"
cosin_similarity_graph = "https://miro.medium.com/v2/resize:fit:1400/format:webp/1*dyH20eCqb6qTL-gt4nCVzQ.png"

# Interface Gradio
demo = gr.Blocks()

with demo:
    gr.Markdown("# Démo IDP - OCR (ici Azure Vision) - Classification par similarité")
    gr.Image(value=diagram_url, label="")

    gr.Interface(
            fn=process_document,
            inputs=[gr.File(label="Fichier", type="filepath", file_types=[".jpg",".pdf",".png",".docx"]),
                    gr.Textbox(label="Ajouter un sous-type de document"),
                    gr.Dropdown(
                        ["Preuve de scolarité", "Billet du médecin", "Spécimen de chèque", "Facture"], label="Dans la catégorie", value="Preuve de scolarité"),
                    gr.CheckboxGroup(["Nettoyage du texte", "Réinitialiser la liste des sous-types/types"], label="Paramètres"),
                    gr.Slider(0.0, 1, value=0.4, label="Seuil")],
            outputs=[gr.Image(label="Image du doc", type="pil"), #gr.Textbox(label="Langue détectée"),
                     gr.Json(label="Classification par similarité (0-Shot classification)"),
                     gr.Textbox(label="Tous les mots extraits (et filtrés)"),
                     gr.Json(label="Tags extraits"),
                     gr.Json(label="Regroupement des sous-types par types (actuellement en mémoire)")],
            title=None,
            examples= [["fac_ortho.jpg"],["billet_medecin.jpg"],["recu_masso_photo.jpg"],["McGill_Enrolment_Letter_2023-2024.pdf"],["releve_notes.pdf"]],
            cache_examples=False, allow_flagging='never', clear_btn=None)

    gr.Image(value=cosin_similarity_graph, label="")

In [ ]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://57f00680b7aa270590.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://57f00680b7aa270590.gradio.live
